### Load our gaming PDFs:

In [ ]:
DATA_PATH = "/teamspace/studios/this_studio/LLM_Courses/Pratiques/RAG+Ollama/data"

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
def load_documents(DATA_PATH):
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [ ]:
Pages = load_documents(DATA_PATH)
Pages

In [ ]:
chunks = split_documents(Pages)
chunks

### Embedding Function

- We should use the same function for creating the database and for embed the prompt !!
- There are alot of models ready to use thanks to langChain

In [ ]:
from langchain_community.embeddings.ollama import OllamaEmbeddings

def Embedding_function():
    embeddings = OllamaEmbeddings(model="nomic-embed-text",show_progress =True)
    return embeddings

### Create Chroma DataBase:

In [ ]:
import os
import shutil
from langchain_community.vectorstores import Chroma

In [ ]:
CHROMA_PATH = "Chroma"

In [ ]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
        
def save_to_chroma(chunks,Ids): 
    clear_database()
    # Create a new DB from the chunks.
    db = Chroma.from_documents(chunks, Embedding_function(), persist_directory=CHROMA_PATH,ids=Ids)
    db.persist() # Forcing the Save
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


def Create_Chunks_Ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0
    ids = []
    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Chunk Id
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id
        ids.append(chunk_id)
    return chunks,ids

In [ ]:
chunks,ids = Create_Chunks_Ids(chunks)
chunks[10]

In [ ]:
save_to_chroma(chunks,ids)

### Update The DataBase

- We want a technic that update the Chroma Database whidout recreate it again each time we add or remove a document ??
  - Easy , we will create a unique id for each embedding vector in the database, made from the **Page Source : Page Number : Chunk Index**

In [ ]:
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=Embedding_function())

In [ ]:
def Get_Existing_ids():
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing chunks in DB: {len(existing_ids)}")
    #print(existing_items)
    return existing_ids


def Update_Chroma(Chunks):
    existing_ids = Get_Existing_ids()
    Chunks,_       = Create_Chunks_Ids(Chunks)
    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in Chunks:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new chunks: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


In [ ]:
chunks[0]

In [ ]:
Update_Chroma(chunks)

### Searching for relevent chunks:

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [ ]:
def Create_Prompt(Question):
    # Searching for Relevent Chunks from DataBase
    results = db.similarity_search_with_relevance_scores(Question,k=3)
    context_text = "\n\n---\n\n".join([chunk.page_content for chunk, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    # The Prompt
    prompt = prompt_template.format(context=context_text, question=Question)
    return prompt,results


In [ ]:
prompt,results = Create_Prompt("How do U get out of jail in Monopoly ?")
print(prompt)

### Prompting Qween2:1.5b:

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="qwen2:1.5b")
response = llm.invoke(prompt)
print(response)

In [ ]:
## Sources used by LLM
sources = [chunk.metadata.get("id", None) for chunk, _score in results]
formatted_response = f"Response: {response}\n\nSources: {sources}"
print(formatted_response)

### Quality Of RAG System :

Its depend on :
- The Quality of Information and Documents.
- The Text Spliting technic.
- The Searching technic (KNN, Clustring...)
- The LLM used and the Embedding model.

#### We can Build Tests, and let the LLM evaluate itself :
- Test like Quetion and Expected response,also we can do the negation that mean give the Quetion and a wrong response and see if the evaluation fail.

In [ ]:
EVAL_PROMPT = """
Actual Response: {actual_response}

Expected Response: {expected_response}
---
(Answer with 'true' or 'false') Does the actual response match the expected response? 
"""

In [ ]:
def Create_Evaluation_Prompt(Response,Expected):
    prompt_template = ChatPromptTemplate.from_template(EVAL_PROMPT)
    # The Prompt
    prompt = prompt_template.format(actual_response=Response,expected_response=Expected)
    return prompt

In [ ]:
Tests =  {
    "How much total money does a player start with in Monopoly? (Answer with the number only)": "$1500",
    "How many points does the longest continuous train get in Ticket to Ride? (Answer with the number only)": "10 points",
}
def Evaluation(Tests):
    for question, answer in Tests.items():
        prompt,_ = Create_Prompt(question)
        response = llm.invoke(prompt)
        #print(response+"\n\n\n")
        
        Evaluation_prompt = Create_Evaluation_Prompt(response,answer)
        Evaluation_response = llm.invoke(Evaluation_prompt)
        # Clean the Evaluation response
        Evaluation = Evaluation_response.strip().lower()
        if "true" in Evaluation:
            print("\033[32mTest Passed\033[0m")
        else :
            print("\033[31mTest Failed\033[0m")

In [ ]:
Evaluation(Tests)